# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 3 2023 10:06AM,258622,10,PRF-54608,Bio-PRF,Released
1,Mar 3 2023 10:06AM,258622,10,PRF-54611,Bio-PRF,Released
2,Mar 3 2023 10:06AM,258622,10,PRF-54614,Bio-PRF,Released
3,Mar 3 2023 10:06AM,258622,10,PRF-54617,Bio-PRF,Released
4,Mar 3 2023 10:06AM,258622,10,PRF-54620,Bio-PRF,Released
5,Mar 3 2023 10:06AM,258622,10,PRF-54623,Bio-PRF,Released
6,Mar 3 2023 10:06AM,258622,10,PRF-54630,Bio-PRF,Released
7,Mar 3 2023 10:06AM,258622,10,PRF-54669,Bio-PRF,Released
8,Mar 3 2023 10:06AM,258622,10,PRF-54672,Bio-PRF,Released
9,Mar 3 2023 10:06AM,258622,10,PRF-54674,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
20,258616,Released,2
21,258617,Released,1
22,258618,Released,1
23,258621,Released,1
24,258622,Released,11


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
258616,NaN,2.0
258617,NaN,1.0
258618,NaN,1.0
258621,NaN,1.0
258622,NaN,11.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
258528,0.0,5.0
258530,0.0,5.0
258533,0.0,4.0
258556,0.0,11.0
258557,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
258528,0,5
258530,0,5
258533,0,4
258556,0,11
258557,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,258528,0,5
1,258530,0,5
2,258533,0,4
3,258556,0,11
4,258557,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,258528,,5
1,258530,,5
2,258533,,4
3,258556,,11
4,258557,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 3 2023 10:06AM,258622,10,Bio-PRF
11,Mar 3 2023 9:57AM,258621,19,"GCH Granules USA, Inc."
12,Mar 3 2023 9:45AM,258618,19,"Emersa Waterbox, LLC"
13,Mar 3 2023 9:44AM,258617,19,"Emersa Waterbox, LLC"
14,Mar 3 2023 9:33AM,258615,15,"Alliance Pharma, Inc."
16,Mar 3 2023 9:33AM,258614,15,"Person & Covey, Inc."
22,Mar 3 2023 9:33AM,258556,15,"Alliance Pharma, Inc."
33,Mar 3 2023 9:31AM,258616,10,"Citieffe, Inc."
35,Mar 3 2023 9:23AM,258613,10,ISDIN Corporation
37,Mar 3 2023 9:22AM,258612,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Mar 3 2023 10:06AM,258622,10,Bio-PRF,,11
1,Mar 3 2023 9:57AM,258621,19,"GCH Granules USA, Inc.",,1
2,Mar 3 2023 9:45AM,258618,19,"Emersa Waterbox, LLC",,1
3,Mar 3 2023 9:44AM,258617,19,"Emersa Waterbox, LLC",,1
4,Mar 3 2023 9:33AM,258615,15,"Alliance Pharma, Inc.",,2
5,Mar 3 2023 9:33AM,258614,15,"Person & Covey, Inc.",,6
6,Mar 3 2023 9:33AM,258556,15,"Alliance Pharma, Inc.",,11
7,Mar 3 2023 9:31AM,258616,10,"Citieffe, Inc.",,2
8,Mar 3 2023 9:23AM,258613,10,ISDIN Corporation,,2
9,Mar 3 2023 9:22AM,258612,10,ISDIN Corporation,,14


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 3 2023 10:06AM,258622,10,Bio-PRF,11,
1,Mar 3 2023 9:57AM,258621,19,"GCH Granules USA, Inc.",1,
2,Mar 3 2023 9:45AM,258618,19,"Emersa Waterbox, LLC",1,
3,Mar 3 2023 9:44AM,258617,19,"Emersa Waterbox, LLC",1,
4,Mar 3 2023 9:33AM,258615,15,"Alliance Pharma, Inc.",2,
5,Mar 3 2023 9:33AM,258614,15,"Person & Covey, Inc.",6,
6,Mar 3 2023 9:33AM,258556,15,"Alliance Pharma, Inc.",11,
7,Mar 3 2023 9:31AM,258616,10,"Citieffe, Inc.",2,
8,Mar 3 2023 9:23AM,258613,10,ISDIN Corporation,2,
9,Mar 3 2023 9:22AM,258612,10,ISDIN Corporation,14,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 3 2023 10:06AM,258622,10,Bio-PRF,11,
1,Mar 3 2023 9:57AM,258621,19,"GCH Granules USA, Inc.",1,
2,Mar 3 2023 9:45AM,258618,19,"Emersa Waterbox, LLC",1,
3,Mar 3 2023 9:44AM,258617,19,"Emersa Waterbox, LLC",1,
4,Mar 3 2023 9:33AM,258615,15,"Alliance Pharma, Inc.",2,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Mar 3 2023 10:06AM,258622,10,Bio-PRF,11.0,NaN
1,Mar 3 2023 9:57AM,258621,19,"GCH Granules USA, Inc.",1.0,NaN
2,Mar 3 2023 9:45AM,258618,19,"Emersa Waterbox, LLC",1.0,NaN
3,Mar 3 2023 9:44AM,258617,19,"Emersa Waterbox, LLC",1.0,NaN
4,Mar 3 2023 9:33AM,258615,15,"Alliance Pharma, Inc.",2.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 3 2023 10:06AM,258622,10,Bio-PRF,11.0,0.0
1,Mar 3 2023 9:57AM,258621,19,"GCH Granules USA, Inc.",1.0,0.0
2,Mar 3 2023 9:45AM,258618,19,"Emersa Waterbox, LLC",1.0,0.0
3,Mar 3 2023 9:44AM,258617,19,"Emersa Waterbox, LLC",1.0,0.0
4,Mar 3 2023 9:33AM,258615,15,"Alliance Pharma, Inc.",2.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3103194,42.0,2.0
12,258599,1.0,0.0
15,775785,19.0,0.0
19,1810045,35.0,19.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3103194,42.0,2.0
1,12,258599,1.0,0.0
2,15,775785,19.0,0.0
3,19,1810045,35.0,19.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,42.0,2.0
1,12,1.0,0.0
2,15,19.0,0.0
3,19,35.0,19.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,42.0
1,12,Released,1.0
2,15,Released,19.0
3,19,Released,35.0
4,10,Executing,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19
Status,,,,
Executing,2.0,0.0,0.0,19.0
Released,42.0,1.0,19.0,35.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19
0,Executing,2.0,0.0,0.0,19.0
1,Released,42.0,1.0,19.0,35.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19
0,Executing,2.0,0.0,0.0,19.0
1,Released,42.0,1.0,19.0,35.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()